In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [2]:
# general imports
import numpy as np
import pandas as pd
from math import ceil
from scipy import linalg
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import make_spd_matrix
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.linalg import toeplitz
from matplotlib import pyplot

In [3]:
# import an optimizer
from scipy.optimize import minimize

In [4]:
class SQRTLasso:
    def __init__(self, alpha=0.01):
        self.alpha = alpha
    
    def fit(self, x, y):
        alpha=self.alpha
        def f_obj(x,y,beta,alpha):
          n =len(x)
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          output = np.sqrt(1/n*np.sum((y-x.dot(beta))**2)) + alpha*np.sum(np.abs(beta))
          return output
        
        def f_grad(x,y,beta,alpha):
          n=x.shape[0]
          p=x.shape[1]
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          output = np.array((-1/np.sqrt(n))*np.transpose(x).dot(y-x.dot(beta))/np.sqrt(np.sum((y-x.dot(beta))**2))+alpha*np.sign(beta)).flatten()
          return output
        
        def objective(beta):
          return(f_obj(x,y,beta,alpha))

        def gradient(beta):
          return(f_grad(x,y,beta,alpha))
        
        beta0 = np.ones((x.shape[1],1))
        output = minimize(objective, beta0, method='L-BFGS-B', jac=gradient,options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 25,'disp': True})
        beta = output.x
        self.coef_ = beta
        
    def predict(self, x):
        return x.dot(self.coef_)

In [5]:
n = 200
p = 1200

In [ ]:
[1]*7

[1, 1, 1, 1, 1, 1, 1]

In [ ]:
[0]*25

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [12]:
beta_star = np.concatenate(([1]*7,[0]*25,[0.25]*5,[0]*50,[0.7]*15,[0]*1098))

In [ ]:
# what we want to detect is the position of the actual information or "signal"
pos = np.where(beta_star!=0)

In [ ]:
np.array(pos).shape[1]

27

In [ ]:
toeplitz([1,2,3,4])

array([[1, 2, 3, 4],
       [2, 1, 2, 3],
       [3, 2, 1, 2],
       [4, 3, 2, 1]])

In [6]:
# we need toeplitz([1,0.8,0.8**2,0.8**3,0.8**4,...0.8**1199])
v = []
for i in range(p):
  v.append(0.8**i)

In [ ]:
v

In [21]:
mu = [0]*p
sigma = 3.5
# Generate the random samples.
np.random.seed(123)
x = np.random.multivariate_normal(mu, toeplitz(v), size=n) # this where we generate some fake data
#y = X.dot(beta) + sigma*np.random.normal(0,1,[num_samples,1])
y = np.matmul(x,beta_star).reshape(-1,1) + sigma*np.random.normal(0,1,size=(n,1))

In [22]:
y.shape

(200, 1)

In [15]:
model = SQRTLasso(alpha=0.1)

In [23]:
model.fit(x,y)

In [18]:
model.coef_

array([ 1.22459961e+00,  1.52938092e+00,  7.80717575e-01, ...,
       -1.84326301e-07,  8.60941267e-06,  7.35081035e-07])

In [24]:
model.coef_

array([ 1.22459961e+00,  1.52938092e+00,  7.80717575e-01, ...,
       -1.84326301e-07,  8.60941267e-06,  7.35081035e-07])

In [ ]:
# apply Lasso and check how many important variables are recovered
model = Lasso(alpha=0.1,fit_intercept=False,max_iter=10000)

In [ ]:
model.fit(x,y)

Lasso(alpha=0.1, fit_intercept=False, max_iter=10000)

In [ ]:
beta_hat = model.coef_

In [ ]:
beta_hat.shape

(1200,)

In [ ]:
beta_hat

array([ 0.87672602,  2.60465118,  0.16529373, ..., -0.        ,
       -0.        , -0.        ])

In [ ]:
pos_lasso = np.where(beta_hat!=0)

In [ ]:
pos_lasso

(array([   0,    1,    2,    3,    4,    5,    6,    7,   18,   29,   32,
          35,   39,   58,   63,   64,   86,   87,   88,   89,   90,   92,
          94,   95,   96,   97,   99,  100,  101,  108,  133,  138,  143,
         151,  156,  163,  164,  168,  180,  184,  191,  192,  200,  243,
         249,  250,  258,  265,  270,  274,  285,  291,  297,  300,  301,
         314,  356,  367,  374,  382,  391,  401,  413,  459,  469,  475,
         480,  481,  482,  498,  503,  504,  515,  518,  538,  572,  575,
         582,  586,  588,  592,  630,  634,  646,  652,  660,  666,  672,
         684,  695,  717,  723,  724,  733,  734,  745,  758,  769,  770,
         779,  785,  805,  807,  824,  858,  860,  871,  876,  881,  897,
         904,  912,  920,  934,  947,  955,  973,  984,  993, 1003, 1010,
        1012, 1018, 1025, 1026, 1031, 1033, 1037, 1054, 1068, 1093, 1094,
        1134, 1140, 1141, 1142, 1146, 1147, 1159, 1166, 1167, 1187, 1194,
        1196]),)

In [ ]:
np.array(pos_lasso).shape[1]

144

In [ ]:
np.intersect1d(pos,pos_lasso).shape[0]

21